# 텍스트 분류 (뉴스그룹 분류)

Raw Text Data --> technology, sports, fashin... 등으로 classification을 해볼 것이다.  
Data는 사이킷런의 fetch_20newsgroups()를 이용한다.

In [1]:
from sklearn.datasets import fetch_20newsgroups
news_data= fetch_20newsgroups(subset='all',random_state=156)

In [3]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


fetch_20newsgroups()의 filenames라는 key는 API가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉토리와 파일명을 지칭한다.  
다음으로 target 클래스가 어떻게 구성돼 있는지 확인해보자

In [13]:
import pandas as pd
print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


target 클래스는 0~19까지 20개로 구성되어 있다. 또한 target:0 --> alt.ahteism, target:1 --> comp.graphics ... 등으로 되어 있다.  
개별 데이터는 어떻게 구성되어 있을까?

In [23]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

데이터는 다음과 같이 구성되어 있으며 기사 내용뿐 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등 다양한 정보 갖고 있다. 이 중에서 내용을 제외하고 제목 등의 다른 정보는 제거하겠다. 왜냐하면 제목과 소속, 이메일 주소 등의 header, footer 정보들은 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 갖고 있는 경우가 많기 때문이다.

따라서 우리는 순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스그룹에 속하는지 분류할 것이다. remove 파라미터를 이용하면 뉴스그룹 기사의 header, footer 등을 제거할 수 있다. 또한 fetch_20newsgroups()는 subset 파라미터를 이용해 train,test 세트를 분리해 내려받을 수 있다.

In [30]:
from sklearn.datasets import fetch_20newsgroups
# subset=train으로 학습용 데이터만 추출 remove=('headers','footers','quotes')
train_news=fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),random_state=156)

X_train=train_news.data
y_train=train_news.target

# subset=test으로 테스트 데이터만 추출 remove=('headers','footers','quotes')
test_news=fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),random_state=156)

X_test=test_news.data
y_test=test_news.target

print('학습 데이터 크기 {0},테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314,테스트 데이터 크기 7532


## 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가